# FlightAI Chatbot with Function Calling

This notebook demonstrates how to build a chatbot with **function calling** (also called tool use) using OpenAI's API and Gradio for the interface.

## Key Concepts:
- **Function Calling**: The LLM can decide when to call functions based on user queries
- **Tool Loop**: When the LLM wants to call a function, we execute it and send the result back
- **Database Integration**: We store ticket prices in SQLite and let the LLM query/update them

In [146]:

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import sqlite3
import gradio as gr
import json

## 1. Import Required Libraries

In [147]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2" - example model name for Ollama - seems to small for this task
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


## 2. Initialize OpenAI Client

Set up the API connection - you can use either OpenAI or Ollama (local).

In [148]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
You can update the prices if asked - 
"""

## 3. System Message

This defines the chatbot's behavior and personality. It's sent with every request to set the context.

In [ ]:
DB = "prices.db"

# Create the database and prices table if it doesn't exist
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

## 4. Database Setup

Create a SQLite database to store ticket prices. The database persists between sessions.

In [ ]:
def get_ticket_price(city):
    """
    🔧 EXECUTED WHEN TOOL IS CALLED
    Retrieves the price for a specific city from the database.
    """
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

## 5. Tool Functions

These are the actual Python functions that will be executed when the LLM requests them.

**🔧 EXECUTION HAPPENS HERE**: When the LLM decides to use a tool, these functions run on your machine.

In [151]:
get_ticket_price("London")

DATABASE TOOL CALLED: Getting price for London


'Ticket price to London is $799.0'

### Test the get_ticket_price function

In [ ]:
def set_ticket_price(city, price):
    """
    🔧 EXECUTED WHEN TOOL IS CALLED
    Updates or inserts a ticket price for a specific city in the database.
    """
    print(f"DATABASE TOOL CALLED: Setting price for {city} to ${price}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        # UPSERT: Insert or update if city already exists
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', 
                      (city.lower(), price, price))
        conn.commit()

In [153]:
# ticket_prices = {"london":799, "paris": 899, "tokyo": 1420, "sydney": 2999}
# for city, price in ticket_prices.items():
#     set_ticket_price(city, price)

### (Optional) Initialize some default prices
Uncomment to populate the database with initial data.

In [ ]:
def chat(message, history):
    """
    Main chat function that handles the conversation and tool calling loop.
    """
    # Convert Gradio history format to OpenAI message format
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    
    # Build the full message list: system message + history + new user message
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # First API call - LLM decides what to do
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(response.choices[0])
    
    # 🔄 TOOL CALLING LOOP
    # Keep looping while the LLM wants to call functions
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        
        # 🔧 EXECUTE THE TOOLS - This is where our Python functions run!
        responses = handle_tool_calls(message)
        
        # Add the tool call request and results to the conversation
        messages.append(message)
        messages.extend(responses)
        
        # Call the LLM again with the tool results
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    # Return the final response to the user
    return response.choices[0].message.content

## 6. Main Chat Function

This is the core function that handles the conversation loop with function calling.

**The Flow:**
1. User sends a message
2. LLM decides if it needs to call a function
3. If yes → we execute the function and send the result back (loop continues)
4. If no → LLM returns its final response to the user

In [ ]:
# Function schema for getting ticket prices
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# Function schema for setting ticket prices
set_price_function = {
    "name": "set_ticket_price",
    "description": "Set the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "price": {
                "type": "number",
                "description": "The new price for the ticket",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    }
}

## 7. Function Definitions for the LLM

These JSON schemas describe our functions to the LLM so it knows:
- What functions are available
- When to use them
- What parameters they need

**Note:** The LLM only sees these descriptions - it doesn't see the actual Python code!

In [ ]:
# Combine both function definitions into the tools list
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": set_price_function}
]

### Package the functions as tools

The `tools` list is sent to the LLM with each request so it knows what functions it can call.

In [ ]:
def handle_tool_calls(message):
    """
    🔧 CRITICAL FUNCTION - This executes the actual Python functions!
    
    Takes the LLM's tool call requests and executes the corresponding Python functions.
    Returns the results in a format the LLM can understand.
    """
    responses = []
    
    # Loop through all tool calls the LLM requested
    for tool_call in message.tool_calls:
        
        # Handle get_ticket_price function
        if tool_call.function.name == "get_ticket_price":
            # Parse the JSON arguments from the LLM
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            
            # 🔧 EXECUTE: Call the actual Python function
            price_details = get_ticket_price(city)
            
            # Format the response for the LLM
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
        
        # Handle set_ticket_price function
        if tool_call.function.name == "set_ticket_price":
            # Parse the JSON arguments from the LLM
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price = arguments.get('price')
            
            # 🔧 EXECUTE: Call the actual Python function
            set_ticket_price(city, price)
            
            # Format the response for the LLM
            responses.append({
                "role": "tool",
                "content": f"Price for {city} updated to ${price}",
                "tool_call_id": tool_call.id
            })
    
    return responses

## 8. Tool Execution Handler

**🔧 THIS IS WHERE THE MAGIC HAPPENS!**

When the LLM decides to call a function:
1. It sends back a `tool_call` with the function name and arguments
2. We parse the arguments from JSON
3. We execute the actual Python function
4. We return the result back to the LLM

This function maps the LLM's requests to our actual Python functions.

In [158]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7891
* To create a public link, set `share=True` in `launch()`.


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_Hlz00ToXNtHZkEnoxbGpW7k5', function=Function(arguments='{"destination_city":"brisbane","price":500}', name='set_ticket_price'), type='function')]))
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_cW7oSRgyUe6PxaiqbknYJDI8', function=Function(arguments='{"destination_city":"brisbane"}', name='get_ticket_price'), type='function')]))
DATABASE TOOL CALLED: Getting price for brisbane
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, 

## 9. Launch the Gradio Interface

This creates a web-based chat interface. Try asking:
- "How much is a ticket to London?"
- "Set the price for Paris to $950"
- "What's the price to Tokyo?"